# <center>Image Colorization Using CNN</center>


## Importing libraries


In [ ]:
import os
import cv2
import keras
import numpy
import skimage
import matplotlib
import tensorflow

## Getting image from dataset and converting into array


In [ ]:
X = []
for imagename in os.listdir('Dataset/Training_image/'):
    X.append(tensorflow.keras.utils.img_to_array(tensorflow.keras.utils.load_img(
        'Dataset/Training_image/'+imagename, target_size=(256, 256))))
X = numpy.array(X, dtype=float)

## Setting Training and Testing data


In [ ]:
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain
Xtest = X[split:]
Xtest = 1.0/255*Xtest

## Defining CNN Model


In [ ]:
model = keras.models.Sequential()

### Input layer


In [ ]:
model.add(keras.layers.Conv2D(64, (3, 3), input_shape=(
    256, 256, 1), activation='relu', padding='same'))

### Hidden Layers


In [ ]:
model.add(keras.layers.Conv2D(
    64, (3, 3), activation='relu', padding='same', strides=2))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(
    128, (3, 3), activation='relu', padding='same', strides=2))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(
    256, (3, 3), activation='relu', padding='same', strides=2))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu',
          padding='same'))
model.add(keras.layers.Conv2D(512, (3, 3), activation='relu',
          padding='same'))
model.add(keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.UpSampling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.UpSampling2D((2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(keras.layers.UpSampling2D((2, 2)))

### Compile model


In [ ]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])

## Image transformation


In [ ]:
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True)

## Generate training data


In [ ]:
batch_size = 10


def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = skimage.color.rgb2lab(batch)
        X_batch = lab_batch[:, :, :, 0]
        Y_batch = lab_batch[:, :, :, 1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

## Train model


In [ ]:
tensorboard = keras.callbacks.TensorBoard(
    log_dir="/content/drive/MyDrive/Colab Notebooks/Image_Colorization/")
trainedmodel = model.fit(image_a_b_gen(batch_size), callbacks=[
                         tensorboard], epochs=500, steps_per_epoch=30)

## Show model accuracy


In [ ]:
matplotlib.pyplot.plot(trainedmodel.history['accuracy'])
matplotlib.pyplot.title('model accuracy')
matplotlib.pyplot.ylabel('accuracy')
matplotlib.pyplot.xlabel('epoch')
matplotlib.pyplot.legend(['train', 'test'], loc='upper left')
matplotlib.pyplot.show()

## Show model loss


In [ ]:
matplotlib.pyplot.plot(trainedmodel.history['loss'])
matplotlib.pyplot.title('model loss')
matplotlib.pyplot.ylabel('loss')
matplotlib.pyplot.xlabel('epoch')
matplotlib.pyplot.legend(['train', 'test'], loc='upper left')
matplotlib.pyplot.show()

## Train and Test data comparison


In [ ]:
matplotlib.pyplot.figure(figsize=(10, 5))

matplotlib.pyplot.subplot(1, 2, 1)
matplotlib.pyplot.hist(Xtrain.flatten(), bins=50,
                       color='blue', alpha=0.7, label='Train Data')
matplotlib.pyplot.title('Train Data Distribution')
matplotlib.pyplot.xlabel('Pixel Intensity')
matplotlib.pyplot.ylabel('Frequency')
matplotlib.pyplot.legend()

matplotlib.pyplot.subplot(1, 2, 2)
matplotlib.pyplot.hist(Xtest.flatten(), bins=50,
                       color='red', alpha=0.7, label='Test Data')
matplotlib.pyplot.title('Test Data Distribution')
matplotlib.pyplot.xlabel('Pixel Intensity')
matplotlib.pyplot.ylabel('Frequency')
matplotlib.pyplot.legend()

matplotlib.pyplot.tight_layout()
matplotlib.pyplot.show()

## Save model


In [ ]:
model_json = model.to_json()
with open("Model/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("Model/model.h5")

## Load model and weight


In [ ]:
json_file = open('Model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights("Model/model.h5")

## Model summery


In [ ]:
loaded_model.summary()

## Process test image


In [ ]:
loaded_model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
Xtest = skimage.color.rgb2lab(1.0/255*X[split:])[:, :, :, 0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = skimage.color.rgb2lab(1.0/255*X[split:])[:, :, :, 1:]
Ytest = Ytest / 128
print(loaded_model.evaluate(Xtest, Ytest, batch_size=10))

## Colorize Image


In [ ]:

# Path to the test folder
test_folder_path = 'Dataset/Testing_Image/'

# Create subplots for displaying original and colorized images
fig, ax = matplotlib.pyplot.subplots(len(os.listdir(
    test_folder_path)), 2, figsize=(16, 4 * len(os.listdir(test_folder_path))))

row = 0
colorize = []

# Iterate over each image in the test folder
for filename in os.listdir(test_folder_path):
    img = cv2.imread(os.path.join(test_folder_path, filename))

    # Check if image is read correctly
    if img is None:
        print(f"Couldn't read image {filename}. Skipping.")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img, (256, 256))
    colorize.append(img_resized)

    ax[row, 0].imshow(img_resized, interpolation='nearest')
    ax[row, 0].set_title(f'Original - {filename}')

    row += 1

# Convert colorize to numpy array and preprocess for testing
colorize = numpy.array(colorize, dtype=float)
colorize = skimage.color.rgb2lab(1.0 / 255 * colorize)[:, :, :, 0]
colorize = colorize.reshape(colorize.shape + (1,))

# Test the model on the colorize images
output = loaded_model.predict(colorize)
output *= 128

row = 0

# Output colorizations
for i in range(len(output)):
    cur = numpy.zeros((256, 256, 3))
    cur[:, :, 0] = colorize[i][:, :, 0]
    cur[:, :, 1:] = output[i]
    resImage = skimage.color.lab2rgb(cur)

    ax[row, 1].imshow(resImage, interpolation='nearest')
    ax[row, 1].set_title(f'Colorized - {filename}')
    row += 1

matplotlib.pyplot.show()